In [1]:
import pandas as pd
import numpy as np
data_aligned_b1 = pd.read_excel('data_aligned_b1.xlsx',
     engine='openpyxl', index_col=0)
data_aligned_b1

,ДатаПробыКокса,M25,M10,CSR,ДатаШихтовки,% спек,% марка Ж,% кокс.,"Оборот печей, ч","Помол, %",...,Vt,I,Io,SI,КТЦ эксп.,MF,MF_spec,MF_otosh,CSR_carb,"Оборот печей отсеч, ч"
0,2017-01-05,88.7,7.7,NaN,2017-01-01,50.0,29.8,28,19.50,77.9,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
1,2017-01-06,88.7,7.5,NaN,2017-01-02,50.0,29.8,28,19.50,77.0,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
2,2017-01-07,88.6,7.6,NaN,2017-01-03,50.0,29.8,28,19.50,76.6,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
3,2017-01-08,88.1,7.6,NaN,2017-01-04,50.0,29.8,28,19.50,76.1,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
4,2017-01-09,88.5,7.8,60.7,2017-01-05,50.0,29.8,28,19.50,77.6,...,62.14549,21.998627,0.20525,5.810,0.7408,14116.40,14095.0,21.40,41.660,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,2021-10-15,87.3,7.8,60.5,2021-10-11,46.0,35.4,50,18.00,74.8,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1708,2021-10-16,87.1,7.8,NaN,2021-10-12,46.0,35.4,50,20.25,74.1,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1709,2021-10-17,87.4,8.0,NaN,2021-10-13,46.0,35.4,50,18.00,75.0,...,41.39000,20.040000,0.14960,3.785,0.5964,5402.95,5250.0,152.95,40.782,18.0
1710,2021-10-18,87.2,7.6,NaN,2021-10-14,46.0,35.4,50,20.75,75.4,...,44.07000,21.080000,0.15976,3.585,0.6208,5364.75,5250.0,114.75,44.298,18.0


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [3]:
results = pd.DataFrame(columns=['model', 'description', 'R2', 'MAE', 'MAPE', 'MSE', 'RMSE', 'Accuracy'])

def evaluate(model, test_features, test_labels):
    
    predictions = model.predict(test_features)
    
    R2 = model.score(test_features, test_labels)
    MAE = metrics.mean_absolute_error(test_labels, predictions)
    MAPE = np.mean(np.abs((test_labels - predictions) / np.abs(test_labels)))
    MSE = metrics.mean_squared_error(test_labels, predictions)
    RMSE = np.sqrt(metrics.mean_squared_error(test_labels, predictions))
    
    feature_importances = pd.DataFrame({'features':list(test_features.columns), 
                                       'feature_importances':  model.feature_importances_})\
                        .sort_values(by='feature_importances', ascending=False)
    
    print('Model Perfomance')
    print('Test R2: {:0.4f}'.format(R2))
    print('MA Error: {:0.4f}'.format(MAE))
    print('MAP Error: {:0.4f}'.format(MAPE))
    print('MS Error: {:0.4f}'.format(MSE))
    print('RMS Error: {:0.4f}'.format(RMSE))
    print('Accuracy:', round(100*(1 - MAPE), 4))
    print(feature_importances)
    results.loc[len(results.index)] = [model, '', R2, MAE, MAPE, MSE, RMSE, round(100*(1 - MAPE), 4)]    

In [4]:
#M25 - target, 22 cols, 20% test
X_1 = data_aligned_b1.iloc[:,5:27]
y_1 = data_aligned_b1['M25']
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2, random_state=0)

In [59]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_1_train, y_1_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 140}

In [5]:
#model_1
model_1 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1.fit(X_1_train, y_1_train)

evaluate(model_1, X_1_test, y_1_test)

Model Perfomance
Test R2: 0.5138
MA Error: 0.2124
MAP Error: 0.0024
MS Error: 0.0729
RMS Error: 0.2700
Accuracy: 99.7621
           features  feature_importances
13               Vt             0.177998
12                σ             0.147834
20         MF_otosh             0.083679
9       Мд. Серы, %             0.073630
15               Io             0.065306
1         % марка Ж             0.049669
14                I             0.044159
5           Пыль, %             0.037773
3   Оборот печей, ч             0.037654
11               R0             0.031299
8         Летуч., %             0.026683
4          Помол, %             0.024791
18               MF             0.024774
21         CSR_carb             0.023617
0            % спек             0.022754
16               SI             0.022722
2           % кокс.             0.022076
19          MF_spec             0.020190
7           Зола, %             0.019942
6          Влага, %             0.018351
17        КТЦ эксп

In [6]:
#model_1 with reduced 6 weak features
X_1_rf = X_1.drop(['Пласт. слой, мм', 'КТЦ эксп.', 'Влага, %', 'Зола, %', 'MF_spec', '% кокс.'], axis=1)

X_1_rf_train, X_1_rf_test, y_1_rf_train, y_1_rf_test = train_test_split(X_1_rf, y_1, test_size=0.2, random_state=0)

model_1_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1_rf.fit(X_1_rf_train, y_1_rf_train)

evaluate(model_1_rf, X_1_rf_test, y_1_rf_test)

Model Perfomance
Test R2: 0.5218
MA Error: 0.2098
MAP Error: 0.0024
MS Error: 0.0717
RMS Error: 0.2677
Accuracy: 99.765
           features  feature_importances
9                Vt             0.185846
8                 σ             0.151875
14         MF_otosh             0.095827
6       Мд. Серы, %             0.077835
11               Io             0.069939
1         % марка Ж             0.054497
10                I             0.050324
13               MF             0.047560
4           Пыль, %             0.042546
2   Оборот печей, ч             0.040353
7                R0             0.037245
5         Летуч., %             0.031950
3          Помол, %             0.030361
12               SI             0.028010
15         CSR_carb             0.028005
0            % спек             0.027829


In [7]:
#model_1 with reduced 12 weak features
X_1_rf_2 = X_1_rf.drop(['% спек', 'CSR_carb', 'SI', 'Помол, %', 'Летуч., %', 'R0'], axis=1)

X_1_rf_2_train, X_1_rf_2_test, y_1_rf_2_train, y_1_rf_2_test = train_test_split(X_1_rf_2, y_1, test_size=0.2, random_state=0)

model_1_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_1_rf_2.fit(X_1_rf_2_train, y_1_rf_2_train)

evaluate(model_1_rf_2, X_1_rf_2_test, y_1_rf_2_test)

Model Perfomance
Test R2: 0.5241
MA Error: 0.2092
MAP Error: 0.0023
MS Error: 0.0713
RMS Error: 0.2671
Accuracy: 99.7657
          features  feature_importances
5               Vt             0.214444
4                σ             0.172587
9         MF_otosh             0.119397
7               Io             0.091920
3      Мд. Серы, %             0.086173
6                I             0.071106
0        % марка Ж             0.070201
8               MF             0.063626
2          Пыль, %             0.060370
1  Оборот печей, ч             0.050176


In [8]:
#M25 - target, 22 cols, 20% test
X_2 = data_aligned_b1.iloc[:,5:28].drop(['Оборот печей, ч'], axis=1)
y_2 = data_aligned_b1['M25']
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.2, random_state=0)

In [68]:

rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_2_train, y_2_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 110}

In [9]:
#model_2
model_2 = RandomForestRegressor(n_estimators=110,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2.fit(X_2_train, y_2_train)

evaluate(model_2, X_2_test, y_2_test)

Model Perfomance
Test R2: 0.5131
MA Error: 0.2129
MAP Error: 0.0024
MS Error: 0.0730
RMS Error: 0.2702
Accuracy: 99.7616
                 features  feature_importances
12                     Vt             0.182069
11                      σ             0.147507
19               MF_otosh             0.087763
8             Мд. Серы, %             0.074501
14                     Io             0.067285
1               % марка Ж             0.055534
13                      I             0.045647
4                 Пыль, %             0.037237
10                     R0             0.030395
0                  % спек             0.027324
17                     MF             0.027279
3                Помол, %             0.026872
7               Летуч., %             0.026820
15                     SI             0.024315
20               CSR_carb             0.023170
2                 % кокс.             0.022607
18                MF_spec             0.020642
6                 Зола, %        

In [10]:
#model_2 with reduced 6 weak features
X_2_rf = X_2.drop(['Пласт. слой, мм', 'Оборот печей отсеч, ч', 'КТЦ эксп.', 'Влага, %', 'Зола, %', 'MF_spec'], axis=1)

X_2_rf_train, X_2_rf_test, y_2_rf_train, y_2_rf_test = train_test_split(X_2_rf, y_2, test_size=0.2, random_state=0)

model_2_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2_rf.fit(X_2_rf_train, y_2_rf_train)

evaluate(model_2_rf, X_2_rf_test, y_2_rf_test)

Model Perfomance
Test R2: 0.5167
MA Error: 0.2116
MAP Error: 0.0024
MS Error: 0.0724
RMS Error: 0.2692
Accuracy: 99.763
       features  feature_importances
9            Vt             0.186644
8             σ             0.153107
14     MF_otosh             0.094744
6   Мд. Серы, %             0.078518
11           Io             0.075820
1     % марка Ж             0.059398
10            I             0.050505
13           MF             0.046771
4       Пыль, %             0.042247
7            R0             0.037551
5     Летуч., %             0.032603
3      Помол, %             0.031136
15     CSR_carb             0.028876
0        % спек             0.028554
12           SI             0.028209
2       % кокс.             0.025318


In [11]:
#model_2 with reduced 12 weak features
X_2_rf_2 = X_2_rf.drop(['% кокс.', 'SI', '% спек', 'CSR_carb', 'Помол, %', 'Летуч., %'], axis=1)

X_2_rf_2_train, X_2_rf_2_test, y_2_rf_2_train, y_2_rf_2_test = train_test_split(X_2_rf_2, y_2, test_size=0.2, random_state=0)

model_2_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_2_rf_2.fit(X_2_rf_2_train, y_2_rf_2_train)

evaluate(model_2_rf_2, X_2_rf_2_test, y_2_rf_2_test)

Model Perfomance
Test R2: 0.5276
MA Error: 0.2088
MAP Error: 0.0023
MS Error: 0.0708
RMS Error: 0.2661
Accuracy: 99.7662
      features  feature_importances
5           Vt             0.206497
4            σ             0.167644
9     MF_otosh             0.123841
7           Io             0.093972
2  Мд. Серы, %             0.087055
0    % марка Ж             0.079034
6            I             0.067393
8           MF             0.064548
1      Пыль, %             0.060036
3           R0             0.049979


In [12]:
#M10 - target, 22 cols, 20% test
X_3 = data_aligned_b1.iloc[:,5:27]
y_3 = data_aligned_b1['M10']
X_3_train, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.2, random_state=0)

In [74]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_3_train, y_3_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 140}

In [13]:
#model_3
model_3 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=4,
                                min_samples_split=2, 
                                random_state=0)
model_3.fit(X_3_train, y_3_train)

evaluate(model_3, X_3_test, y_3_test)

Model Perfomance
Test R2: 0.2107
MA Error: 0.1779
MAP Error: 0.0250
MS Error: 0.0528
RMS Error: 0.2298
Accuracy: 97.5043
           features  feature_importances
12                σ             0.163220
20         MF_otosh             0.093429
14                I             0.069432
15               Io             0.062710
9       Мд. Серы, %             0.060666
6          Влага, %             0.049793
5           Пыль, %             0.048575
1         % марка Ж             0.047817
4          Помол, %             0.040577
11               R0             0.037611
8         Летуч., %             0.036894
21         CSR_carb             0.033187
17        КТЦ эксп.             0.031590
7           Зола, %             0.030857
13               Vt             0.029999
16               SI             0.029664
0            % спек             0.029206
2           % кокс.             0.026294
18               MF             0.024974
3   Оборот печей, ч             0.024414
19          MF_spe

In [14]:
#model_3 with reduced 6 weak features
X_3_rf = X_3.drop(['Пласт. слой, мм', 'MF_spec', 'Оборот печей, ч', 'MF', '% кокс.', '% спек'], axis=1)

X_3_rf_train, X_3_rf_test, y_3_rf_train, y_3_rf_test = train_test_split(X_3_rf, y_3, test_size=0.2, random_state=0)

model_3_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_3_rf.fit(X_3_rf_train, y_3_rf_train)

evaluate(model_3_rf, X_3_rf_test, y_3_rf_test)

Model Perfomance
Test R2: 0.1998
MA Error: 0.1795
MAP Error: 0.0252
MS Error: 0.0535
RMS Error: 0.2314
Accuracy: 97.4809
       features  feature_importances
8             σ             0.162243
14     MF_otosh             0.095746
10            I             0.076875
11           Io             0.073359
6   Мд. Серы, %             0.064791
2       Пыль, %             0.062834
0     % марка Ж             0.062601
3      Влага, %             0.058294
1      Помол, %             0.050126
5     Летуч., %             0.048164
7            R0             0.043477
15     CSR_carb             0.041775
4       Зола, %             0.041213
13    КТЦ эксп.             0.040943
12           SI             0.039090
9            Vt             0.038467


In [15]:
#model_3 with reduced 12 weak features
X_3_rf_2 = X_3_rf.drop(['Vt', 'SI', 'КТЦ эксп.', 'Зола, %', 'CSR_carb', 'R0'], axis=1)

X_3_rf_2_train, X_3_rf_2_test, y_3_rf_2_train, y_3_rf_2_test = train_test_split(X_3_rf_2, y_3, test_size=0.2, random_state=0)

model_3_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_3_rf_2.fit(X_3_rf_2_train, y_3_rf_2_train)

evaluate(model_3_rf_2, X_3_rf_2_test, y_3_rf_2_test)

Model Perfomance
Test R2: 0.1865
MA Error: 0.1814
MAP Error: 0.0254
MS Error: 0.0544
RMS Error: 0.2333
Accuracy: 97.4561
      features  feature_importances
6            σ             0.194290
9     MF_otosh             0.132462
8           Io             0.114483
7            I             0.112946
2      Пыль, %             0.084563
0    % марка Ж             0.082941
5  Мд. Серы, %             0.081750
3     Влага, %             0.073042
1     Помол, %             0.063926
4    Летуч., %             0.059597


In [16]:
#M10 - target, 22 cols, 20% test
X_4 = data_aligned_b1.iloc[:,5:28].drop(['Оборот печей, ч'], axis=1)
y_4 = data_aligned_b1['M10']
X_4_train, X_4_test, y_4_train, y_4_test = train_test_split(X_4, y_4, test_size=0.2, random_state=0)

In [79]:
rf = RandomForestRegressor(random_state=0)

parameters = {'n_estimators': range(10, 150, 10),
             'max_depth': range(2, 12, 2),
             'min_samples_leaf': range(2, 12, 2),
             'min_samples_split': range(2, 12, 2)
             }

grid = GridSearchCV(rf, parameters, n_jobs=-1, cv=5)
grid.fit(X_4_train, y_4_train)
grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 140}

In [17]:
#model_4
model_4 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=4,
                                min_samples_split=2, 
                                random_state=0)
model_4.fit(X_4_train, y_4_train)

evaluate(model_4, X_4_test, y_4_test)

Model Perfomance
Test R2: 0.2093
MA Error: 0.1781
MAP Error: 0.0250
MS Error: 0.0529
RMS Error: 0.2300
Accuracy: 97.5006
                 features  feature_importances
11                      σ             0.163754
19               MF_otosh             0.094615
13                      I             0.069647
14                     Io             0.064814
8             Мд. Серы, %             0.061544
4                 Пыль, %             0.050030
5                Влага, %             0.050010
1               % марка Ж             0.048520
3                Помол, %             0.041542
10                     R0             0.037833
7               Летуч., %             0.037046
20               CSR_carb             0.034731
16              КТЦ эксп.             0.032833
12                     Vt             0.031496
15                     SI             0.031259
6                 Зола, %             0.030516
0                  % спек             0.029015
2                 % кокс.        

In [18]:
#model_4 with reduced 6 weak features
X_4_rf = X_4.drop(['Пласт. слой, мм', 'Оборот печей отсеч, ч', 'MF_spec', 'MF', '% кокс.', '% спек'], axis=1)

X_4_rf_train, X_4_rf_test, y_4_rf_train, y_4_rf_test = train_test_split(X_4_rf, y_4, test_size=0.2, random_state=0)

model_4_rf = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_4_rf.fit(X_4_rf_train, y_4_rf_train)

evaluate(model_4_rf, X_4_rf_test, y_4_rf_test)

Model Perfomance
Test R2: 0.1998
MA Error: 0.1795
MAP Error: 0.0252
MS Error: 0.0535
RMS Error: 0.2314
Accuracy: 97.4809
       features  feature_importances
8             σ             0.162243
14     MF_otosh             0.095746
10            I             0.076875
11           Io             0.073359
6   Мд. Серы, %             0.064791
2       Пыль, %             0.062834
0     % марка Ж             0.062601
3      Влага, %             0.058294
1      Помол, %             0.050126
5     Летуч., %             0.048164
7            R0             0.043477
15     CSR_carb             0.041775
4       Зола, %             0.041213
13    КТЦ эксп.             0.040943
12           SI             0.039090
9            Vt             0.038467


In [19]:
#model_4 with reduced 12 weak features
X_4_rf_2 = X_4_rf.drop(['Vt', 'SI', 'КТЦ эксп.', 'Зола, %', 'CSR_carb', 'R0'], axis=1)

X_4_rf_2_train, X_4_rf_2_test, y_4_rf_2_train, y_4_rf_2_test = train_test_split(X_4_rf_2, y_4, test_size=0.2, random_state=0)

model_4_rf_2 = RandomForestRegressor(n_estimators=140,
                                max_depth=10,
                                min_samples_leaf=2,
                                min_samples_split=2, 
                                random_state=0)
model_4_rf_2.fit(X_4_rf_2_train, y_4_rf_2_train)

evaluate(model_4_rf_2, X_4_rf_2_test, y_4_rf_2_test)

Model Perfomance
Test R2: 0.1865
MA Error: 0.1814
MAP Error: 0.0254
MS Error: 0.0544
RMS Error: 0.2333
Accuracy: 97.4561
      features  feature_importances
6            σ             0.194290
9     MF_otosh             0.132462
8           Io             0.114483
7            I             0.112946
2      Пыль, %             0.084563
0    % марка Ж             0.082941
5  Мд. Серы, %             0.081750
3     Влага, %             0.073042
1     Помол, %             0.063926
4    Летуч., %             0.059597


In [20]:
results.description = ['M25_22_Оборот печей, ч', 'M25_16_Оборот печей, ч', 'M25_10_Оборот печей, ч', 
                       'M25_22_Оборот печей отсеч, ч', 'M25_16_Оборот печей отсеч, ч', 'M25_10_Оборот печей отсеч, ч',
                       'M10_22_Оборот печей, ч', 'M10_16_Оборот печей, ч', 'M10_10_Оборот печей, ч', 
                       'M10_22_Оборот печей отсеч, ч', 'M10_16_Оборот печей отсеч, ч', 'M10_10_Оборот печей отсеч, ч']
results

,model,description,R2,MAE,MAPE,MSE,RMSE,Accuracy
0,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_22_Оборот печей, ч",0.513811,0.212397,0.002379,0.072881,0.269965,99.7621
1,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_16_Оборот печей, ч",0.521757,0.209823,0.002350,0.071690,0.267750,99.7650
2,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_10_Оборот печей, ч",0.524093,0.209185,0.002343,0.071340,0.267095,99.7657
3,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_22_Оборот печей отсеч, ч",0.513103,0.212890,0.002384,0.072987,0.270161,99.7616
4,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_16_Оборот печей отсеч, ч",0.516730,0.211647,0.002370,0.072444,0.269153,99.7630
5,"(DecisionTreeRegressor(max_depth=10, max_featu...","M25_10_Оборот печей отсеч, ч",0.527584,0.208774,0.002338,0.070816,0.266114,99.7662
6,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_22_Оборот печей, ч",0.210737,0.177886,0.024957,0.052810,0.229805,97.5043
7,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_16_Оборот печей, ч",0.199778,0.179482,0.025191,0.053544,0.231395,97.4809
8,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_10_Оборот печей, ч",0.186500,0.181358,0.025439,0.054432,0.233307,97.4561
9,"(DecisionTreeRegressor(max_depth=10, max_featu...","M10_22_Оборот печей отсеч, ч",0.209335,0.178147,0.024994,0.052904,0.230009,97.5006
